<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AAI_DEMO_LIBRARY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_openai langchain_core langchain_google_genai -q

# Install necessary libraries and nest_asyncio
!pip install -q langchain openai crewai autogen langgraph pydantic-ai mistralai google-generativeai langchain-google-genai langchain-community nest_asyncio
!pip install -q "pydantic_ai>=0.2.0" # Ensure latest PydanticAI if previous version gave issues

## LangChain: Basic Agent with a Tool

In [2]:
# langchain_demo.py
import os
from google.colab import userdata
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI # For Gemini
from langchain_core.tools import tool

# --- Configure API Keys from Colab Secrets ---
os.environ["GOOGLE_API_KEY"] = userdata.get('GEMINI')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

print("--- LangChain Demo ---")

# 1. Define the LLM (Using Gemini as per project goal - UPDATED MODEL NAME)
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0) # Changed from gemini-pro

# 2. Define Tools
@tool
def get_current_time(location: str = "EST") -> str:
    """Gets the current time for a given location, defaults to EST."""
    import datetime
    from zoneinfo import ZoneInfo

    try:
        if location.upper() == "EST":
            tz = ZoneInfo("America/New_York")
        else:
            tz = ZoneInfo("America/New_York")
            print(f"Warning: Only EST time is supported in this mock tool. Providing EST for {location}.")

        now_est = datetime.datetime.now(tz)
        return f"The current time in {location.upper()} is {now_est.strftime('%Y-%m-%d %H:%M:%S %Z%z')}"
    except Exception as e:
        return f"Could not get time for {location}: {e}"

tools = [get_current_time]

# 3. Create the Agent Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful flight planning assistant. You have access to tools. Use them when appropriate."),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

# 4. Create the Agent
agent = create_tool_calling_agent(llm, tools, prompt)

# 5. Create the Agent Executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 6. Run the Agent
async def run_langchain_agent():
    print("Asking LangChain agent: 'What is the current time in EST?'")
    result = await agent_executor.ainvoke({"input": "What is the current time in EST?", "chat_history": []})
    print("\nLangChain Agent Response:")
    print(result["output"])

    print("\nAsking LangChain agent: 'What is the capital of France?' (Note: Will not be able to answer without a general search tool)")
    result = await agent_executor.ainvoke({"input": "What is the capital of France?", "chat_history": []})
    print("\nLangChain Agent Response:")
    print(result["output"])

await run_langchain_agent()

--- LangChain Demo ---
Asking LangChain agent: 'What is the current time in EST?'


> Entering new AgentExecutor chain...

Invoking: `get_current_time` with `{}`


The current time in EST is 2025-07-20 23:53:49 EDT-0400The current time in EST is 2025-07-20 23:53:49 EDT-0400. 

> Finished chain.

LangChain Agent Response:
The current time in EST is 2025-07-20 23:53:49 EDT-0400. 

Asking LangChain agent: 'What is the capital of France?' (Note: Will not be able to answer without a general search tool)


> Entering new AgentExecutor chain...
The capital of France is Paris.

> Finished chain.

LangChain Agent Response:
The capital of France is Paris.


## Microsoft AutoGen: Conversational Agents (Colab Secrets for OpenAI)

In [4]:
# autogen_demo.py
import autogen
import os
from google.colab import userdata

# --- Configure API Keys from Colab Secrets ---
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

print("\n--- AutoGen Demo ---")

# 1. Configuration for the LLM
config_list = [
    {
        "model": "gpt-4o",
        "api_key": os.environ["OPENAI_API_KEY"],
    },
    {
        "model": "gpt-4o-mini",
        "api_key": os.environ["OPENAI_API_KEY"],
    },
]

# 2. Create the Assistant Agent
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "seed": 42,
        "config_list": config_list,
        "temperature": 0
    },
    system_message="You are a helpful AI assistant. You can write and execute Python code.",
)

# 3. Create the User Proxy Agent
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={"work_dir": "coding", "use_docker": False},
)

# 4. Define the task and initiate chat
async def run_autogen_agents():
    print("Initiating AutoGen chat: 'Plot a chart of y = x^2 for x from -5 to 5. Save it as 'parabola.png'.'")
    await user_proxy.a_initiate_chat( # This is the async call
        assistant,
        message="Plot a chart of y = x^2 for x from -5 to 5 using matplotlib. Save it as 'parabola.png'. Include the code and confirm once done. TERMINATE",
    )
    print("\nAutoGen Chat Complete.")

# Create a 'coding' directory if it doesn't exist for code execution
if not os.path.exists("coding"):
    os.makedirs("coding")

# In Colab, after applying nest_asyncio, directly await the async function
await run_autogen_agents()


--- AutoGen Demo ---
Initiating AutoGen chat: 'Plot a chart of y = x^2 for x from -5 to 5. Save it as 'parabola.png'.'
user_proxy (to assistant):

Plot a chart of y = x^2 for x from -5 to 5 using matplotlib. Save it as 'parabola.png'. Include the code and confirm once done. TERMINATE

--------------------------------------------------------------------------------
assistant (to user_proxy):

To plot the chart of \( y = x^2 \) for \( x \) ranging from -5 to 5 using matplotlib and save it as 'parabola.png', you can use the following code:

```python
import matplotlib.pyplot as plt
import numpy as np

# Define the range of x values
x = np.linspace(-5, 5, 100)

# Define the function y = x^2
y = x**2

# Create the plot
plt.figure()
plt.plot(x, y, label='y = x^2')
plt.title('Plot of y = x^2')
plt.xlabel('x')
plt.ylabel('y')
plt.grid(True)
plt.legend()

# Save the plot as a PNG file
plt.savefig('parabola.png')

# Show the plot
plt.show()
```

I will execute this code to generate and save the

## CrewAI: Multi-Agent Collaboration

In [1]:
# crewai_demo.py
import os
from google.colab import userdata
from crewai import Agent, Task, Crew, Process, LLM # Import CrewAI's LLM class

# --- Configure API Keys from Colab Secrets ---
# Retrieve the API key explicitly
GEMINI_API_KEY_VALUE = userdata.get('GEMINI')

# For good measure, set GOOGLE_API_KEY environment variable as well
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY_VALUE

# For OpenAI (if you still want to test with it):
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

print("\n--- CrewAI Demo ---")

# 1. Define LLM for CrewAI using CrewAI's own LLM class (which wraps LiteLLM)
# This format explicitly passes the api_key and is reported to work for Gemini.
llm = LLM(
    model='gemini/gemini-2.5-flash', # Model with provider prefix
    temperature=0.2,
    api_key=GEMINI_API_KEY_VALUE # <--- Pass the API key directly to CrewAI's LLM
)

# 2. Define Agents
researcher = Agent(
    role='Senior Research Analyst',
    goal='Uncover interesting facts about the history of aviation.',
    backstory='A skilled analyst with a knack for discovering hidden gems of information.',
    verbose=True,
    allow_delegation=False,
    llm=llm # <--- Pass the configured CrewAI LLM instance
)

writer = Agent(
    role='Professional Content Writer',
    goal='Craft engaging and informative articles based on research findings',
    backstory='A wordsmith who transforms raw data into compelling narratives.',
    verbose=True,
    allow_delegation=False,
    llm=llm # <--- Pass the configured CrewAI LLM instance
)

# 3. Define Tasks
task_research = Task(
    description='Conduct a thorough research on the history of aviation.',
    agent=researcher,
    expected_output='A comprehensive report outlining key milestones and figures in aviation history.'
)

task_write = Task(
    description='Write a 500-word engaging article based on the research findings about aviation history.',
    agent=writer,
    context=[task_research],
    expected_output='A well-structured, engaging 500-word article on aviation history.'
)

# 4. Form the Crew
crew = Crew(
    agents=[researcher, writer],
    tasks=[task_research, task_write],
    verbose=False,
    process=Process.sequential
)

# 5. Kick off the Crew's work
print("Starting CrewAI for aviation history research and writing...")
result = crew.kickoff()

print("\nCrewAI Output:")
print(result)


--- CrewAI Demo ---
Starting CrewAI for aviation history research and writing...


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Task: Conduct a thorough research on the history of aviation.                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **A Comprehensive Report on the History of Aviation: Key Milestones and Figures**                              │
│                                                                                                                 │
│  The history of aviation is a testament to human ingenuity, perseverance, and the enduring dream of flight.     │
│  From ancient myths of winged gods to the supersonic jets of today, the journey has been marked by              │
│  groundbreaking discoveries, daring pioneers, and relentless technological advancement.                         │
│                                                                                                                 │
│  **I. Early Concepts and Pre-Flight Attempts (Pre-19th Century)**                                               │
│                                                                                                                 │
│  Long before the first successful flight, humanity dreamed of soaring through the skies.                        │
│  *   **Ancient Times:** Myths and legends, such as Icarus and Daedalus, reflect early aspirations for flight.   │
│  *   **Leonardo da Vinci (Late 15th - Early 16th Century):** The Renaissance polymath meticulously studied      │
│  bird flight and produced detailed designs for various flying machines, including ornithopters (wing-flapping   │
│  machines), helicopters (his "aerial screw" is a precursor), and parachutes. While his designs were centuries   │
│  ahead of available technology and power sources, they laid theoretical groundwork.                             │
│  *   **Montgolfier Brothers (Joseph-Michel and Jacques-Étienne Montgolfier) (1783):** These French brothers     │
│  achieved the first successful human flight. On September 19, 1783, their hot air balloon carried a sheep, a    │
│  duck, and a rooster as passengers. On November 21, 1783, Jean-François Pilâtre de Rozier and François          │
│  Laurent, Marquis d'Arlandes, made the first untethered human flight in Paris, covering about 5.5 miles in 25   │
│  minutes. This marked the dawn of lighter-than-air flight.                                                      │
│                                                                                                                 │
│  **II. The Pioneering Age of Heavier-Than-Air Flight (19th Century - Early 20th Century)**                      │
│                                                                                                                 │
│  The 19th century saw the scientific principles of aerodynamics begin to take shape, leading to practical       │
│  designs.                                                                                                       │
│  *   **Sir George Cayley (Late 18th - Mid 19th Century):** Often called the "Father of Aeronautics," Cayley     │
│  was the first to understand and define the four fundamental forces of flight: lift, drag, thrust, and weight.  │
│  He designed and flew successful gliders, including one in 1853 that carried his coachman (who reportedly quit  │
│  his job afterward, stating he was "hired to drive, not to fly"). Cayley's work established the concept of the  │
│  fixed-wing aircraft.                                                                                           │
│  *   **Otto Lilienthal (1890s):** This German engineer 

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Professional Content Writer                                                                             │
│                                                                                                                 │
│  Task: Write a 500-word engaging article based on the research findings about aviation history.                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Professional Content Writer                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Soaring Through Time: A Journey Through Aviation History**                                                   │
│                                                                                                                 │
│  From mythical wings to roaring jets, humanity's quest for flight is a saga of relentless innovation and        │
│  unwavering determination. Aviation history isn't just a timeline of machines; it's a testament to the          │
│  boundless human spirit, transforming an ancient dream into a global reality.                                   │
│                                                                                                                 │
│  Centuries before powered flight, visionaries like Leonardo da Vinci meticulously sketched flying machines,     │
│  laying theoretical groundwork. The true dawn of aerial travel, however, arrived with the Montgolfier brothers  │
│  in 1783, whose hot air balloons lifted the first humans into the sky, marking the beginning of                 │
│  lighter-than-air flight.                                                                                       │
│                                                                                                                 │
│  The 19th century shifted focus to heavier-than-air principles. Sir George Cayley, the "Father of               │
│  Aeronautics," defined the fundamental forces of flight, designing successful gliders. Otto Lilienthal's        │
│  systematic glider experiments further refined understanding of control. Yet, the ultimate breakthrough came    │
│  from two American bicycle mechanics, Orville and Wilbur Wright. On December 17, 1903, at Kitty Hawk, North     │
│  Carolina, their "Flyer I" achieved the first sustained, controlled, powered flight. This monumental            │
│  achievement, born from meticulous research and a revolutionary three-axis control system, truly launched       │
│  practical aviation. Soon, pioneers like Alberto Santos-Dumont and Louis Blériot, who famously crossed the      │
│  English Channel in 1909, demonstrated flight's growing potential.                                              │
│                                                                                                                 │
│  World War I dramatically accelerated aircraft development, transforming planes from novelties into vital       │
│  military assets. The interwar period saw aviation transition to commercial use. Airmail services flourished,   │
│  and daring "barnstormers" captivated audiences. Charles Lindbergh's solo non-stop transatlantic flight in      │
│  1927 in the "Spirit of St. Louis" captivated the world, igniting public fascination and investment. Amelia     │
│  Earhart further pushed boundaries. Crucially, the Douglas DC-3, introduced in 1935, revolutionized commercial  │
│  travel, becoming the first airliner profitable solely on passenger fares, making air travel accessible.        │
│                                                                                                                 │
│  World War II spurred unprecedented technological leaps, directly leading to the jet age. Independently         │
│  developed by Hans von Ohain and Frank Whittle, the jet engine powered the first jet aircraft flights.          │
│  Post-war, Chuck Yeager shattered the sound barrier in 


CrewAI Output:
**Soaring Through Time: A Journey Through Aviation History**

From mythical wings to roaring jets, humanity's quest for flight is a saga of relentless innovation and unwavering determination. Aviation history isn't just a timeline of machines; it's a testament to the boundless human spirit, transforming an ancient dream into a global reality.

Centuries before powered flight, visionaries like Leonardo da Vinci meticulously sketched flying machines, laying theoretical groundwork. The true dawn of aerial travel, however, arrived with the Montgolfier brothers in 1783, whose hot air balloons lifted the first humans into the sky, marking the beginning of lighter-than-air flight.

The 19th century shifted focus to heavier-than-air principles. Sir George Cayley, the "Father of Aeronautics," defined the fundamental forces of flight, designing successful gliders. Otto Lilienthal's systematic glider experiments further refined understanding of control. Yet, the ultimate breakthro

## LangGraph: Simple State Machine Agent

In [2]:
# langgraph_demo.py
import operator
from typing import Annotated, TypedDict
from google.colab import userdata
import os

from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI # For Gemini
from langgraph.graph import StateGraph, START, END

# --- Configure API Keys from Colab Secrets ---
os.environ["GOOGLE_API_KEY"] = userdata.get('GEMINI')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

print("\n--- LangGraph Demo ---")

# 1. Define Agent State
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

# 2. Define the LLM (Using Gemini as per project goal - UPDATED MODEL NAME)
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0) # Changed from gemini-pro

# 3. Define Nodes (functions for different steps)
def call_llm(state: AgentState):
    messages = state['messages']
    response = llm.invoke(messages)
    return {"messages": [response]}

def decide_to_greet(state: AgentState):
    last_message = state['messages'][-1].content
    if "hello" in last_message.lower() or "hi" in last_message.lower():
        return "greet_user"
    else:
        return "ask_question"

def greet_user(state: AgentState):
    return {"messages": [AIMessage(content="Hello there! How can I assist you today?")]}

def ask_question(state: AgentState):
    return {"messages": [AIMessage(content="I see. What can I help you with?")]}

# 4. Build the Graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("call_llm", call_llm)
workflow.add_node("greet_user", greet_user)
workflow.add_node("ask_question", ask_question)

# Set entry point
workflow.set_entry_point("call_llm")

# Add edges
workflow.add_conditional_edges(
    "call_llm",
    decide_to_greet,
    {
        "greet_user": "greet_user",
        "ask_question": "ask_question",
    },
)

workflow.add_edge("greet_user", END)
workflow.add_edge("ask_question", END)

# Compile the graph
app = workflow.compile()

# 5. Run the graph
async def run_langgraph_agent(query):
    print(f"\nRunning LangGraph with input: '{query}'")
    inputs = {"messages": [HumanMessage(content=query)]}
    async for output in app.astream(inputs):
        for key, value in output.items():
            print(f"Output from node '{key}': {value}")
    print("LangGraph Execution Complete.")

await run_langgraph_agent("Hello, agent!")
await run_langgraph_agent("What is the weather today?")


--- LangGraph Demo ---

Running LangGraph with input: 'Hello, agent!'
Output from node 'call_llm': {'messages': [AIMessage(content="Hello there! I'm here and ready to help.\n\nWhat can I do for you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--abc9e8b6-8e8b-4e52-b3d3-4ac83086f4bb-0', usage_metadata={'input_tokens': 5, 'output_tokens': 21, 'total_tokens': 1045, 'input_token_details': {'cache_read': 0}})]}
Output from node 'greet_user': {'messages': [AIMessage(content='Hello there! How can I assist you today?', additional_kwargs={}, response_metadata={})]}
LangGraph Execution Complete.

Running LangGraph with input: 'What is the weather today?'
Output from node 'call_llm': {'messages': [AIMessage(content='I cannot give you the real-time weather for your specific location because I don\'t know where you are. As an AI, I don\'t have access to your current location or 

## PydanticAI: Structured Output Agent

In [9]:
# pydanticai_demo.py
import os
from google.colab import userdata
from pydantic import BaseModel, Field
# Corrected import for Agent and Tool
from pydantic_ai import Agent, Tool
from pydantic_ai.models.gemini import GeminiModel
from pydantic_ai.providers.google_gla import GoogleGLAProvider
from typing import List, Callable, Any # Import Callable and Any for type hinting

# --- Configure API Keys from Colab Secrets ---
GEMINI_API_KEY_VALUE = userdata.get('GEMINI')
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY_VALUE

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

print("\n--- PydanticAI Demo ---")

# 1. Define a Pydantic Model for the expected output
class FlightInfo(BaseModel):
    flight_number: str = Field(description="The unique flight identifier (e.g., 'DL123').")
    origin_airport: str = Field(description="The ICAO or IATA code of the departure airport (e.g., 'JFK').")
    destination_airport: str = Field(description="The ICAO or IATA code of the arrival airport (e.g., 'LAX').")
    departure_time_utc: str = Field(description="The scheduled departure time in UTC (e.g., '2025-07-20T14:30:00Z').")
    status: str = Field(description="The current status of the flight (e.g., 'On Time', 'Delayed', 'Cancelled').")
    duration_hours: float = Field(description="The estimated flight duration in hours.")

# 2. Define the tool function itself (UPDATED - no explicit input_model/output_model args)
# PydanticAI infers the schema from the function's type hints.
async def mock_flight_lookup(flight_number: str) -> FlightInfo:
    """Looks up details for a specific flight number."""
    print(f"DEBUG: Mocking flight lookup for: {flight_number}")
    return FlightInfo(
        flight_number=flight_number.upper().replace(" ", ""),
        origin_airport="JFK",
        destination_airport="LAX",
        departure_time_utc="2025-07-21T08:00:00Z",
        status="On Time",
        duration_hours=5.5
    )

# 3. Create the Tool instance by wrapping the function (UPDATED - removed input_model/output_model)
flight_lookup_tool_instance = Tool(
    function=mock_flight_lookup,
    name="flight_lookup",
    description="Looks up details for a specific flight number."
)

# 4. Initialize the Agent with an LLM and Tool
async def run_pydanticai_agent():
    google_provider = GoogleGLAProvider(api_key=GEMINI_API_KEY_VALUE)

    gemini_llm = GeminiModel(
        "gemini-2.5-flash",
        provider=google_provider
    )

    agent = Agent(
        model=gemini_llm,
        # description="You are a flight information assistant. Use the 'flight_lookup' tool to get flight details.", # Removed unsupported argument
        tools=[flight_lookup_tool_instance], # Pass the instance of the Tool
        # verbose=True # Removed unsupported argument
    )

    print("Asking PydanticAI agent to find flight DL456:")
    result = await agent.run("Find details for flight DL456.")

    print("\nPydanticAI Agent Output (Structured):")
    if isinstance(result, FlightInfo):
        print(f"Flight Number: {result.flight_number}")
        print(f"Origin: {result.origin_airport}")
        print(f"Destination: {result.destination_airport}")
        print(f"Departure: {result.departure_time_utc}")
        print(f"Status: {result.status}")
        print(f"Duration: {result.duration_hours} hours")
    else:
        print(result)

await run_pydanticai_agent()


--- PydanticAI Demo ---
Asking PydanticAI agent to find flight DL456:
DEBUG: Mocking flight lookup for: DL456

PydanticAI Agent Output (Structured):
AgentRunResult(output='Flight DL456, operated by Delta, is scheduled to depart from JFK at 8:00 AM UTC on July 21, 2025, and arrive at LAX after a 5.5-hour flight. It is currently on time.')


## OpenAI Agents SDK: Multi-Agent Handoff (Colab Secrets for OpenAI)

In [ ]:
# Ensure openai-agents is installed for the OpenAI Agents SDK demo
!pip install -q openai-agents
!pip install -q colab-env

In [2]:
from IPython import get_ipython
from IPython.display import display
import asyncio
import os
import random
import re
import pandas as pd
from agents import Agent, Tool, Runner # Assuming 'agents' library is correctly installed
from sklearn.linear_model import LogisticRegression
from openai import OpenAI, AsyncOpenAI
import colab_env # For Google Colab environment variables


# Initialize AsyncOpenAI client
client = AsyncOpenAI(api_key=os.environ.get("OPENAI_API_KEY"))


class FlightEnvironment:
    def __init__(self, origin, destination, weather):
        self.origin = origin
        self.destination = destination
        self.weather = weather

    def get_possible_routes(self):
        routes = {
            ("New York", "London"): [
                {
                    "flight_number": "AC888",
                    "departure_time": "20:00",
                    "arrival_time": "08:00",
                    "price": 800,
                    "airline": "Air Canada",
                    "aircraft_type": "Boeing 787-9",
                },
                {
                    "flight_number": "UA901",
                    "departure_time": "22:00",
                    "arrival_time": "10:00",
                    "price": 900,
                    "airline": "United Airlines",
                    "aircraft_type": "Boeing 777-300ER",
                },
            ]
        }
        return routes.get((self.origin, self.destination), [])

    def get_weather_data(self, location):
        weather_conditions = ["Sunny", "Cloudy", "Rainy", "Overcast", "Snowy"]
        wind_speed = random.randint(5, 20)
        return {
            "weather": [{"description": random.choice(weather_conditions)}],
            "wind": {"speed": wind_speed},
        }

    def update(self):
        print("Updating environment...")
        self.weather = {
            self.origin: self.get_weather_data(self.origin),
            self.destination: self.get_weather_data(self.destination),
        }


class FlightAgent(Agent):
    def __init__(self, environment, turbulence_dataset_path, **kwargs):
        super().__init__(
            name="FlightPlanner",
            model="gpt-4o",
            instructions="You are a flight planning expert. Generate a detailed and safe flight plan based on the provided information.",
            **kwargs,
        )

        self.environment = environment
        self.turbulence_model = LogisticRegression()
        self.turbulence_dataset_path = turbulence_dataset_path
        self.train_turbulence_model()
        self.prompt = None # Initialize prompt to None

    def train_turbulence_model(self):
        try:
            turbulence_df = pd.read_csv(self.turbulence_dataset_path)
            turbulence_df["Turbulence"] = (turbulence_df["REF_k"] > turbulence_df["REF_k"].mean()).astype(int)
            target = turbulence_df["Turbulence"]
            features = turbulence_df[
                [
                    "REF_U_1",
                    "REF_U_2",
                    "REF_U_3",
                    "REF_tau_11",
                    "REF_tau_12",
                    "REF_tau_13",
                    "REF_tau_22",
                    "REF_tau_23",
                    "REF_tau_33",
                ]
            ]
            self.turbulence_model.fit(features, target)
            print("Turbulence model trained successfully!")
        except FileNotFoundError:
            print(f"Error: Turbulence dataset not found at {self.turbulence_dataset_path}. Skipping turbulence model training.")
        except Exception as e:
            print(f"An error occurred during turbulence model training: {e}")

    async def run(self, max_iterations=5):
        """Executes the OODA loop for the FlightAgent."""
        for iteration in range(max_iterations):
            print(f"\n--- OODA Loop Iteration {iteration + 1} ---")
            observations = {
                "origin_weather": self.environment.get_weather_data(self.environment.origin),
                "destination_weather": self.environment.get_weather_data(self.environment.destination),
                "possible_routes": self.environment.get_possible_routes(),
            }
            print("Observations:", observations)
            situation_summary = f"""
            Current Situation:
            - Origin Weather: {observations['origin_weather']['weather'][0]['description']}
            - Destination Weather: {observations['destination_weather']['weather'][0]['description']}
            - Available Routes: {observations['possible_routes']}
            """
            print(situation_summary)

            # Generate flight plan using OpenAIAgent's call_model (Corrected)
            flight_plan = await self.generate_flight_plan(observations)

            print("Flight Plan:", flight_plan)
            print("Simulating flight execution...")
            self.environment.update()

    async def generate_flight_plan(self, observations):
        origin_weather = self.environment.get_weather_data(self.environment.origin)
        destination_weather = self.environment.get_weather_data(self.environment.destination)
        possible_routes = self.environment.get_possible_routes()

        print('\n\n')
        print(f"Weather in {self.environment.origin}: {origin_weather['weather'][0]['description']}")
        print(f"Weather in {self.environment.destination}: {destination_weather['weather'][0]['description']}")
        print("Available flights:")
        print('\n')
        print(f"Origin: {self.environment.origin}")
        print(f"Destination: {self.environment.destination}")
        print('\n')
        for route in possible_routes:
            print(route)

        if not possible_routes:
            return "No flights found."

        """Generates a flight plan using the embedded OpenAI model."""
        prompt = f"""
        Generate a detailed flight plan for a long-haul flight from {self.environment.origin} to {self.environment.destination},
        considering the following:

        ## Recommended Flight and Aircraft Type:
        - Consider these available flights: {observations['possible_routes']}
        - Only consider flights operated by Air Canada (AC) or United Airlines (UA).
        - Based on the available flights, recommend the most suitable flight and provide a justification for your choice.
        - **Clearly state the aircraft type for the recommended flight (e.g., Boeing 777, Airbus A330).**

        ## Route and Waypoints:
        - Determine and include a list of suitable waypoints for the recommended flight, using standard aviation codes (e.g., KJFK, EGLL).
        - Consider typical North Atlantic Tracks (NATs) for optimal routing and fuel efficiency.
        - The origin and destination airport codes are: {self.environment.origin} and {self.environment.destination}.

        ## Fuel Calculations:
        - **Provide a detailed breakdown of fuel requirements for different phases of the flight (e.g., taxi, takeoff, climb, cruise, descent, landing).**
        - Include the total fuel required for the trip.

        ## ETOPS Considerations:
        - State the ETOPS certification for the recommended aircraft.
        - List suitable ETOPS alternate airports.

        ## Risks and Mitigation Strategies:
        - **Provide detailed explanations of potential risks associated with this specific flight and aircraft type, along with specific mitigation strategies.**
        - Consider risks such as crew fatigue, jet lag, clear air turbulence, decompression, and medical emergencies.
        - Take into account the current weather conditions: {observations['origin_weather']['weather'][0]['description']} at the origin and {observations['destination_weather']['weather'][0]['description']} at the destination.

        ## Additional Information for Pilot and Crew:
        - Provide any relevant information for the pilot and crew, such as NOTAMs, airspace restrictions, oceanic procedures, and potential delays.
        """

        self.prompt = prompt  # Set the prompt attribute here

        response = await client.chat.completions.create(  # Use client initialized with AsyncOpenAI
            model="gpt-4o",  # or "gpt-3.5-turbo"
            messages=[{"role": "user", "content": prompt}],
        )
        flight_plan = response.choices[0].message.content.strip()

        # Post-processing (if needed)
        flight_plan = re.sub(r"\n+", "\n", flight_plan)
        flight_plan = re.sub(r"## (.*)", r"\n\n**\1**:", flight_plan)

        print('\n')
        print('Flight plan is running ....')
        print('\n\n')
        print("Flight Plan:", flight_plan)
        print('\n')

        return flight_plan


# Example usage
environment = FlightEnvironment(
    origin="New York",
    destination="London",
    weather={
        "New York": {"weather": [{"description": "Clear"}], "wind": {"speed": 5}},
        "London": {"weather": [{"description": "Overcast"}], "wind": {"speed": 10}},
    },
)


flight_agent = FlightAgent(environment, "/content/gdrive/MyDrive/datasets/turbulence/REF.csv")


async def main():
    """Executes the main logic of the flight agent."""
    # This is the correct way to execute the OODA loop as designed in FlightAgent.
    # It will perform observations, generate the flight plan, and simulate environment updates.
    await flight_agent.run(max_iterations=1)


import nest_asyncio

nest_asyncio.apply()  # Apply nest_asyncio to enable nested event loops

if __name__ == "__main__":
    print("Initializing Flight Planning Agent (gemini 2.0)")
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

Turbulence model trained successfully!
Initializing Flight Planning Agent (gemini 2.0)

--- OODA Loop Iteration 1 ---
Observations: {'origin_weather': {'weather': [{'description': 'Sunny'}], 'wind': {'speed': 5}}, 'destination_weather': {'weather': [{'description': 'Cloudy'}], 'wind': {'speed': 13}}, 'possible_routes': [{'flight_number': 'AC888', 'departure_time': '20:00', 'arrival_time': '08:00', 'price': 800, 'airline': 'Air Canada', 'aircraft_type': 'Boeing 787-9'}, {'flight_number': 'UA901', 'departure_time': '22:00', 'arrival_time': '10:00', 'price': 900, 'airline': 'United Airlines', 'aircraft_type': 'Boeing 777-300ER'}]}

            Current Situation:
            - Origin Weather: Sunny
            - Destination Weather: Cloudy
            - Available Routes: [{'flight_number': 'AC888', 'departure_time': '20:00', 'arrival_time': '08:00', 'price': 800, 'airline': 'Air Canada', 'aircraft_type': 'Boeing 787-9'}, {'flight_number': 'UA901', 'departure_time': '22:00', 'arrival_time':

## Mistral AI Agents API: Basic Agent with Web Search Tool (Colab Secrets for Mistral)

In [24]:

try:
    import mistralai
    # Re-import after upgrade to ensure new version is loaded in current session if needed
    from mistralai import Mistral
except ImportError as e:
    print(f"Error importing Mistral AI SDK components: {e}")
    print("Please ensure 'mistralai' package is correctly installed and up-to-date.")
    print("If the error persists, please restart your Python runtime/kernel after running 'pip install mistralai --upgrade'.")
    exit()

# %%
# Ensure MISTRAL_API_KEY is set up
api_key = userdata.get('MISTRAL_API_KEY')

if not api_key:
    print("Error: MISTRAL_API_KEY environment variable not set.")
    print("Please set your Mistral API key before running this script.")
    exit()

client = Mistral(api_key=api_key)

# --- Agent Definitions ---
# Pydantic model for Calculator Agent's response format (still relevant if that agent is used)
class CalcResult(BaseModel):
    reasoning: str
    result: str

print("Creating AI agents...")

finance_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent used to answer financial related requests",
    name="finance-agent",
)
web_search_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent that can search online for any information if needed",
    name="websearch-agent",
    tools=[{"type": "web_search"}],
)
ecb_interest_rate_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Can find the current interest rate of the European central bank",
    name="ecb-interest-rate-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_european_central_bank_interest_rate",
                "description": "Retrieve the real interest rate of European central bank.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "date": {
                            "type": "string",
                        },
                    },
                    "required": [
                        "date",
                    ]
                },
            },
        },
    ],
)
graph_agent = client.beta.agents.create(
    model="mistral-large-latest",
    name="graph-drawing-agent",
    description="Agent used to create graphs using the code interpreter tool.",
    instructions="Use the code interpreter tool when you have to draw a graph.",
    tools=[{"type": "code_interpreter"}]
)
calculator_agent = client.beta.agents.create(
    model="mistral-large-latest",
    name="calculator-agent",
    description="Agent used to make detailed calculations",
    instructions="When doing calculations explain step by step what you are doing.",
    completion_args={
          "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "calc_result",
                "schema": CalcResult.model_json_schema(),
            }
        }
    }
)

# --- Custom function for flight search (MOCK) ---
# This is a LOCAL MOCK FUNCTION.
# When the remote Mistral AI model proposes to use this tool,
# this local Python function needs to be executed by your application.
# For a real integration, this functionality would typically be
# an actual web service endpoint accessible by Mistral's platform.
def search_flights(origin: str, destination: str, departure_date: str, return_date: str = None):
    """
    MOCK: This function simulates searching for available flights.
    It returns dummy data based on specific input parameters.
    """
    print(f"\n[DEBUG] MOCK CALL: search_flights with origin='{origin}', destination='{destination}', departure_date='{departure_date}', return_date='{return_date}')")

    origin_norm = origin.upper() if origin else ""
    destination_norm = destination.upper() if destination else ""

    # Dummy data for demonstration
    if origin_norm == "YUL" and destination_norm == "JFK" and departure_date == "2025-07-01":
        return {
            "flights_found": True,
            "details": [
                {"flight_number": "AC700", "airline": "Air Canada", "departure_time": "10:00 AM", "arrival_time": "11:30 AM", "price": "$250 CAD"},
                {"flight_number": "DL123", "airline": "Delta Airlines", "departure_time": "11:00 AM", "arrival_time": "12:45 PM", "price": "$280 CAD"}
            ]
        }
    elif origin_norm == "LAX" and destination_norm == "SFO" and departure_date == "2025-08-15":
        return {
            "flights_found": True,
            "details": [
                {"flight_number": "UA456", "airline": "United Airlines", "departure_time": "09:00 AM", "arrival_time": "10:15 AM", "price": "$120 USD"},
            ]
        }
    elif origin_norm == "YUL" and destination_norm in ["HND", "NRT"]: # Assuming HND or NRT for Tokyo
        return {
            "flights_found": True,
            "details": [
                 {"flight_number": "AC005", "airline": "Air Canada", "departure_time": "01:45 PM", "arrival_time": "03:30 PM +1 day", "price": "$1200 CAD"},
                 {"flight_number": "NH117", "airline": "ANA", "departure_time": "04:00 PM", "arrival_time": "05:50 PM +1 day", "price": "$1350 CAD"}
            ]
        }
    # Add new dummy data for a different route/date
    elif origin_norm == "LHR" and destination_norm == "CDG" and departure_date == "2025-09-10":
        return {
            "flights_found": True,
            "details": [
                {"flight_number": "BA123", "airline": "British Airways", "departure_time": "08:00 AM", "arrival_time": "10:30 AM", "price": "£150 GBP"},
                {"flight_number": "AF456", "airline": "Air France", "departure_time": "09:00 AM", "arrival_time": "11:45 AM", "price": "€170 EUR"}
            ]
        }
    else:
        return {"flights_found": False, "details": "No flights found for the specified criteria."}

# --- Flight Planning Agent Definition ---
# The agent is defined with its capabilities and tools on the Mistral platform.
flight_planning_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for assisting with flight planning requests, searching for flights and providing relevant information. Use IATA airport codes for origin and destination if possible. For example, Montreal is YUL and New York is JFK or LGA.",
    name="flight-planning-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "search_flights",
                "description": "Search for available flights based on origin airport code, destination airport code, and departure date. Returns flight details if found.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "origin": {"type": "string", "description": "Departure airport IATA code (e.g., YUL for Montreal, LAX for Los Angeles)"},
                        "destination": {"type": "string", "description": "Arrival airport IATA code (e.g., JFK for New York, SFO for San Francisco)"},
                        "departure_date": {"type": "string", "description": "Departure date inYYYY-MM-DD format"},
                        "return_date": {"type": "string", "description": "Return date inYYYY-MM-DD format (optional)"}
                    },
                    "required": ["origin", "destination", "departure_date"]
                }
            }
        },
        # For agent creation, this simple format for web_search is usually correct.
        {"type": "web_search"}
    ]
)

print(f"\nFlight Planning Agent '{flight_planning_agent.name}' created with ID: {flight_planning_agent.id}")
tool_names_list = []
for tool in flight_planning_agent.tools:
    if tool.type == 'function':
        tool_names_list.append(tool.function.name)
    else:
        tool_names_list.append(tool.type)
print(f"Tools available to flight_planning_agent: {tool_names_list}")


# --- Test Case Execution for Flight Planning Agent ---
print("\n--- Executing Test Cases for the Flight Planning Agent (Consecutive) ---")
print("This simulates multi-turn conversations where your code acts as the tool executor.")

# Manual construction of tools list for the chat.complete call (remains the same for both tests)
api_call_tools_list = [
    {
        "type": "function",
        "function": {
            "name": "search_flights",
            "description": "Search for available flights based on origin airport code, destination airport code, and departure date. Returns flight details if found.",
            "parameters": {
                "type": "object",
                "properties": {
                    "origin": {"type": "string", "description": "Departure airport IATA code (e.g., YUL for Montreal, LAX for Los Angeles)"},
                    "destination": {"type": "string", "description": "Arrival airport IATA code (e.g., JFK for New York, SFO for San Francisco)"},
                    "departure_date": {"type": "string", "description": "Departure date inYYYY-MM-DD format"},
                    "return_date": {"type": "string", "description": "Return date inYYYY-MM-DD format (optional)"}
                },
                "required": ["origin", "destination", "departure_date"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "internal_web_search_tool",
            "description": "Accesses the internet to find information.",
            "parameters": {
                "type": "object",
                "properties": {}
            }
        }
    }
]

# --- Test Case 1: YUL to NRT ---
print("\n--- Test Case 1: YUL to NRT on July 10th, 2025 ---")
user_query_1 = "Find me a flight from Montreal (YUL) to Tokyo on July 10th, 2025."
print(f"\nUser: {user_query_1}")
conversation_history_1 = [{"role": "user", "content": user_query_1}]

try:
    print("[DEBUG] Sending initial user query 1 to the agent...")
    response_turn1_1 = client.chat.complete(
        model=flight_planning_agent.model,
        messages=conversation_history_1,
        tools=api_call_tools_list,
    )
    assistant_message_turn1_1 = response_turn1_1.choices[0].message
    conversation_history_1.append(assistant_message_turn1_1.model_dump() if hasattr(assistant_message_turn1_1, 'model_dump') else assistant_message_turn1_1.__dict__)

    if hasattr(assistant_message_turn1_1, 'tool_calls') and assistant_message_turn1_1.tool_calls:
        print("\nAgent proposed tool calls (Turn 1, Test 1):")
        for tool_call in assistant_message_turn1_1.tool_calls:
            print(f"  Tool Name: {tool_call.function.name}")
            print(f"  Tool Arguments (JSON string): {tool_call.function.arguments}")
            tool_output_content = None
            if tool_call.function.name == "search_flights":
                try:
                    args = json.loads(tool_call.function.arguments)
                    tool_output = search_flights(**args)
                    tool_output_content = json.dumps(tool_output)
                    print(f"  [DEBUG] Local MOCK search_flights executed. Output: {tool_output_content}")
                except json.JSONDecodeError as e:
                    print(f"  [ERROR] Failed to parse tool arguments: {e}")
                    tool_output_content = json.dumps({"error": f"Failed to parse arguments: {e}"})
                except Exception as e:
                    print(f"  [ERROR] Error executing local mock search_flights: {e}")
                    tool_output_content = json.dumps({"error": f"Tool execution failed: {e}"})
            elif tool_call.function.name == "internal_web_search_tool":
                print(f"  [DEBUG] Web search requested by agent. Providing mock output for local simulation.")
                tool_output_content = json.dumps("Mock web search: Information related to flight search.")
            else:
                print(f"  [DEBUG] Unhandled tool call: {tool_call.function.name}")
                tool_output_content = json.dumps({"error": "Tool not handled by client-side executor."})

            conversation_history_1.append(
                {
                    "role": "tool",
                    "name": tool_call.function.name,
                    "content": tool_output_content,
                    "tool_call_id": tool_call.id
                }
            )
            print(f"  [DEBUG] Tool output for '{tool_call.function.name}' added to history 1.")

        print("\n[DEBUG] Sending conversation history 1 with tool outputs back for final response...")
        final_response_1 = client.chat.complete(
            model=flight_planning_agent.model,
            messages=conversation_history_1,
            tools=api_call_tools_list,
        )
        final_assistant_message_1 = final_response_1.choices[0].message
        print("\nAgent's Final Response (Test 1):")
        print(final_assistant_message_1.content)
        conversation_history_1.append(final_assistant_message_1.model_dump() if hasattr(final_assistant_message_1, 'model_dump') else assistant_message_turn1_1.__dict__)
    else:
        print("\nAgent's initial response (no tool calls proposed, Test 1):")
        print(assistant_message_turn1_1.content)

except Exception as e:
    print(f"\nAn error occurred during agent interaction (Test 1): {e}")


print("\n--- Test Case 1 Complete ---")

# --- Test Case 2: LHR to CDG ---
print("\n--- Test Case 2: LHR to CDG on September 10th, 2025 ---")
user_query_2 = "Find me a flight from London (LHR) to Paris (CDG) on September 10th, 2025."
print(f"\nUser: {user_query_2}")
conversation_history_2 = [{"role": "user", "content": user_query_2}] # Start new history for Test 2

try:
    print("[DEBUG] Sending initial user query 2 to the agent...")
    response_turn1_2 = client.chat.complete(
        model=flight_planning_agent.model,
        messages=conversation_history_2,
        tools=api_call_tools_list,
    )
    assistant_message_turn1_2 = response_turn1_2.choices[0].message
    conversation_history_2.append(assistant_message_turn1_2.model_dump() if hasattr(assistant_message_turn1_2, 'model_dump') else assistant_message_turn1_2.__dict__)

    if hasattr(assistant_message_turn1_2, 'tool_calls') and assistant_message_turn1_2.tool_calls:
        print("\nAgent proposed tool calls (Turn 1, Test 2):")
        for tool_call in assistant_message_turn1_2.tool_calls:
            print(f"  Tool Name: {tool_call.function.name}")
            print(f"  Tool Arguments (JSON string): {tool_call.function.arguments}")
            tool_output_content = None
            if tool_call.function.name == "search_flights":
                try:
                    args = json.loads(tool_call.function.arguments)
                    tool_output = search_flights(**args)
                    tool_output_content = json.dumps(tool_output)
                    print(f"  [DEBUG] Local MOCK search_flights executed. Output: {tool_output_content}")
                except json.JSONDecodeError as e:
                    print(f"  [ERROR] Failed to parse tool arguments: {e}")
                    tool_output_content = json.dumps({"error": f"Failed to parse arguments: {e}"})
                except Exception as e:
                    print(f"  [ERROR] Error executing local mock search_flights: {e}")
                    tool_output_content = json.dumps({"error": f"Tool execution failed: {e}"})
            elif tool_call.function.name == "internal_web_search_tool":
                print(f"  [DEBUG] Web search requested by agent. Providing mock output for local simulation.")
                tool_output_content = json.dumps("Mock web search: Information related to flight search.")
            else:
                print(f"  [DEBUG] Unhandled tool call: {tool_call.function.name}")
                tool_output_content = json.dumps({"error": "Tool not handled by client-side executor."})

            conversation_history_2.append(
                {
                    "role": "tool",
                    "name": tool_call.function.name,
                    "content": tool_output_content,
                    "tool_call_id": tool_call.id
                }
            )
            print(f"  [DEBUG] Tool output for '{tool_call.function.name}' added to history 2.")


        print("\n[DEBUG] Sending conversation history 2 with tool outputs back for final response...")
        final_response_2 = client.chat.complete(
            model=flight_planning_agent.model,
            messages=conversation_history_2,
            tools=api_call_tools_list,
        )
        final_assistant_message_2 = final_response_2.choices[0].message
        print("\nAgent's Final Response (Test 2):")
        print(final_assistant_message_2.content)
        conversation_history_2.append(final_assistant_message_2.model_dump() if hasattr(final_assistant_message_2, 'model_dump') else final_assistant_message_2.__dict__)
    else:
        print("\nAgent's initial response (no tool calls proposed, Test 2):")
        print(assistant_message_turn1_2.content)

except Exception as e:
    print(f"\nAn error occurred during agent interaction (Test 2): {e}")


print("\n--- Test Case 2 Complete ---")

print("\n--- All Consecutive Test Cases Complete ---")

Creating AI agents...

Flight Planning Agent 'flight-planning-agent' created with ID: ag_01982b682cba7423bb3c95ed44685dcc
Tools available to flight_planning_agent: ['search_flights', 'web_search']

--- Executing Test Cases for the Flight Planning Agent (Consecutive) ---
This simulates multi-turn conversations where your code acts as the tool executor.

--- Test Case 1: YUL to NRT on July 10th, 2025 ---

User: Find me a flight from Montreal (YUL) to Tokyo on July 10th, 2025.
[DEBUG] Sending initial user query 1 to the agent...

Agent proposed tool calls (Turn 1, Test 1):
  Tool Name: search_flights
  Tool Arguments (JSON string): {"origin": "YUL", "destination": "NRT", "departure_date": "2025-07-10"}

An error occurred during agent interaction (Test 1): name 'json' is not defined

--- Test Case 1 Complete ---

--- Test Case 2: LHR to CDG on September 10th, 2025 ---

User: Find me a flight from London (LHR) to Paris (CDG) on September 10th, 2025.
[DEBUG] Sending initial user query 2 to t